# Titanic Notebook

This notebook is based on the excellent [Titanic Tutorial](https://www.kaggle.com/code/alexisbcook/titanic-tutorial) by [Alexis Cook](https://www.kaggle.com/alexisbcook). After running Alexis's code and making the initial submission, I started to explore ways for improving the prediction accuracy.

## Ideas for improving prediction accuracy

**Augmenting missing data.** Titanic's training data is composed of 891 entries and test data is composed of 418 entries. A number of the 891 entries in Titanic's training data are missing some information, most notably:
* the `Age` column, for which only 714 entries are non-null entries, and
* the `Cabin` column, for which only 204 entries are non-null entries.

Presumably &mdash; something we should confirm during the exploratory data analysis &mdash; the age of a passenger impacts their chances of survival. If you're, say, 5 years old or younger, you probably don't know how to swim and if you're 70 years old or older, you probably don't have the stamina to hold on for longer than 15 minutes or so.

Moreover, the cabin of a passenger could also have an effect on their survival chances. According to [Wikipedia article on the sinking of the Titanic](https://en.wikipedia.org/wiki/Sinking_of_the_Titanic), the ship sank at 02:20 (ship's time) on 15 April 1912 after stricking an iceberg at 23:40 on 14 April. If the ship struck the iceberg 20 minutes before midnight, chances are that most passengers were in their cabins. Later on, when the ship sank, it might have been challenging to escape from cabins located in the ship's hull.

Section [Survivors and victims](https://en.wikipedia.org/wiki/Titanic#Survivors_and_victims) in the Wikipedia's article states that the survival rates show stark differences for different classes aboard Titanic:

> Although only 3% of first-class women were lost, 54% of those in third-class died. Similarly, five of six first-class and all second-class children survived, but 52 of the 79 in third-class perished. The differences by gender were even bigger: nearly all female crew members, first- and second-class passengers were saved. Men from the First Class died at a higher rate than women from the Third Class. In total, 50% of the children survived, 20% of the men and 75% of the women.

So, my first idea was to augment the missing data.

**Tuning hyperparameters.** tbd

# Load Data
First things first, let's load the data.

In [1]:
# This Python 3 environment comes with many helpful analytic s libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Exploratory Data Analysis

Let's take a look at the training data and the test data.

In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


The first observation from Alexis tutorial is that the survival rate of women are much higher than that of men:

In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of women who survived:", rate_women)
print("% of men who survived:", rate_men)

% of women who survived: 0.7420382165605095
% of men who survived: 0.18890814558058924


Next, let's check whether we have entries with missing values (`NaN`) in our training data.

In [5]:
# How many NaNs do we have in our training data?
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
# How many NaNs do we have in our test data?
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


So, let's see what the entries with missing age look like.

In [7]:
# Select all passengers where Age is null
#train_data_no_age = train_data[train_data["Age"].isnull()]

#for index, row in train_data_no_age.iterrows():
#    print(f"Index: {index}, Values: {row.values}") # or a custom format

Now that we know that some entries both in training and in test data are missing values for "Age", let's fix it. The idea is to assign the missing age based on the passenger's title. We can do this by taking the median for passengers with different titles (Master, Miss./Ms., Mrs., Mr.)

In [8]:
title_age = {"Master": 0, "Miss.": 0, "Ms.": 0, "Mrs.": 0, "Mr.": 0}

for key in title_age:
    age_data = train_data[(train_data['Name'].str.contains(key, na=False)) & (train_data['Age'].notnull())]
    title_age[key] = age_data["Age"].median()

print(title_age)

{'Master': 3.5, 'Miss.': 21.0, 'Ms.': 28.0, 'Mrs.': 35.0, 'Mr.': 31.0}


In [9]:
for key in title_age:
    #missing_data = train_data[(train_data['Name'].str.contains(key, na=False)) & (train_data['Age'].isnull())]
    train_data[(train_data['Name'].str.contains(key, na=False)) & (train_data['Age'].isnull())].fillna(title_age[key], inplace=True)
    test_data[(test_data['Name'].str.contains(key, na=False)) & (test_data['Age'].isnull())].fillna(title_age[key], inplace=True)
    
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

/tmp/ipykernel_17/1468498274.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data[(train_data['Name'].str.contains(key, na=False)) & (train_data['Age'].isnull())].fillna(title_age[key], inplace=True)
/tmp/ipykernel_17/1468498274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[(train_data['Name'].str.contains(key, na=False)) & (train_data['Age'].isnull())].fillna(title_age[key], inplace=True)
/tmp/ipykernel_17/1468498274.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.i

# Cleaning Data

We want to fill up the `NaN` values with reasonable data.

In [10]:
# Clean the training data
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)

# Clean test data
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)

/tmp/ipykernel_17/2940277791.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
/tmp/ipykernel_17/2940277791.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

y = train_data["Survived"]

features = ["Pclass", "Age", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')

# Display the results
print(f'Cross-Validation Accuracy Scores: {scores}')
print(f'Mean Accuracy: {scores.mean()}')
print(f'Standard Deviation of Accuracy: {scores.std()}')


Cross-Validation Accuracy Scores: [0.83798883 0.81460674 0.80898876 0.80898876 0.85393258]
Mean Accuracy: 0.8249011361496453
Standard Deviation of Accuracy: 0.018036043682637486


Finally, we submit the result


In [12]:
predictions = model.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
